<a href="https://colab.research.google.com/github/kurniadewy/bank_churn_predicition_interpretability/blob/main/EC5_Assign_CustomerSatisfaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Customer Satisfaction and Sentiment Analysis


## Objective

You are a data analyst of a consulting company that provides customer insight regarding multiple ticketing system, such as JIRA and Zoho Desk. Your team gather surveys to customers regarding their ticketing system's performance. Your role in the team is to gather reports regarding customer satisfaction and sentiment analysis into a single dashboard and present your insight.

Analyze the following metrics and other insight you can find in the dataset:

- Survey response rate
- Customer Satisfaction score (CSAT)
- Customer Effort Score (CES)
- Net Promoter Score (NPS)
- Sentiment Analysis



## Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

### Access to Drive

Write where you put the data in google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# where is your data
data_path = '/content/drive/My Drive/Online Course/DIBIMBING/Dataset/'

Mounted at /content/drive/


### Read Data

Read the file **assignment_ticket_system_review.csv**

In [ ]:
# Read Data
df = pd.read_csv(data_path  + 'assignment_ticket_system_review.csv')

df.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif..."
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...


The following is the dictionary for the data, survey is only valid if all of the survey questions and text review is not blank (null):

**General Information**
- id_survey: identifier for each survey
- date_of_survey: date of survey taken
- ticket_system: The name of the ticket system being reviewed (e.g. Zoho Desk)

**Survey Questions**
- overall_rating: The overall satisfaction rating given by the reviewer, ranging from 1 to 5
- customer_service: The satisfaction rating for the customer service provided by the ticket system, ranging from 1 to 5.
- features: The satisfaction rating for the features of the ticket system, ranging from 1 to 5
- value_for_money: The satisfaction rating for the value for money provided by the ticket system, ranging from 1 to 5
- ease_of_use: The rating for how easy the ticket system is to use, ranging from 1 to 5
- likelihood_to_recommend: The likelihood that the reviewer would recommend the ticket system to others, ranging from 1 to 10
- overall_text: The full text of the overall review, providing detailed feedback on the ticket system.


In [ ]:
# Check the type of data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_survey                1462 non-null   object 
 1   date_of_survey           1462 non-null   object 
 2   ticket_system            1462 non-null   object 
 3   overall_rating           787 non-null    float64
 4   customer_service         787 non-null    float64
 5   features                 787 non-null    float64
 6   value_for_money          787 non-null    float64
 7   ease_of_use              787 non-null    float64
 8   likelihood_to_recommend  787 non-null    float64
 9   overall_text             787 non-null    object 
dtypes: float64(6), object(4)
memory usage: 114.3+ KB


### Data Cleansing

Convert the date column into proper date_time format.

In [ ]:
# Convert data type
df['date_of_survey'] = pd.to_datetime(df['date_of_survey'])

df.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif..."
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...


## Survey Analysis

### Response Rate

Start by analyzing how many customers has filled the survey, indicated by whether the overall_rating is not blank.

In [ ]:
# How many customer responded to the survey?
df['fill_survey'] = np.where(df['overall_rating'].isnull(), 'Not Responded', 'Responded')

df.value_counts('fill_survey', normalize = True).reset_index()

,fill_survey,proportion
0,Responded,0.54
1,Not Responded,0.46


Create a new dataframe that consists only of those who have responded the survey to simplify calculating the CSAT, CES, and NPS Score.

In [ ]:
# Responded Customer
responded_customer = df[ df['fill_survey'] == 'Responded'].copy()


### CSAT Score

Measure the customer's overall satisfaction score (CSAT) with the following formula:

$$
CSAT = \frac{\Sigma\ total\ satisfaction\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

The max rating is inserted to convert the CSAT score into percentage.

CSAT score can be classified into categories based on the result. There is no absolute threshold for each categories but the following is the common threshold:

- \>= 90%: Excellent
- 75%-90%: Good
- 60-75%: Fair
- \<60%: Poor

In [ ]:
# CSAT Score
max_rating = 5
n_data = (responded_customer.shape[0] * max_rating)

csat_score = responded_customer['overall_rating'].sum() / n_data

print(f'Overall CSAT Score: { (csat_score * 100):.1f}%')

Overall CSAT Score: 91.2%


Dengan Overall CSAT Score sebesar 91.2%, kepuasan pelanggan terhadap sistem tiket termasuk dalam kategori "Excellent". Ini menunjukkan bahwa sebagian besar pengguna merasa sangat puas dengan layanan yang diberikan.

Measure the satisfaction score for the following attributes:

- customer service
- features
- value for money

In [ ]:
# Satisfaction Score for Attributes
score_customer_service = responded_customer['customer_service'].sum() / n_data
score_features = responded_customer['features'].sum()/ n_data
score_value_for_money = responded_customer['value_for_money'].sum()/ n_data

print(f'Overall CSAT Score: { (csat_score * 100):.1f}%')
print(f'Customer Service: { (score_customer_service * 100):.1f}%')
print(f'Features: { (score_features * 100):.1f}%')
print(f'Value for Money: { (score_value_for_money * 100):.1f}%')

Overall CSAT Score: 91.2%
Customer Service: 67.3%
Features: 88.3%
Value for Money: 87.6%


Dari score yang diperoleh, customer merasa bahwa harga yang dikeluarkan sebanding dengan manfaat yang diperoleh, serta fitur sistem tiket juga sudah cukup memuaskan. Namun, untuk pelayanan terhadap customer perlu ditingkatkan.

### CES Score

Measure CES with the following formula


$$
CES = \frac{\Sigma\ total\ effort\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

In [ ]:
# CES Score
max_rating = 5

ces_ease_of_use = responded_customer['ease_of_use'].sum()/ n_data

print(f'Pick Up CES Score: { (ces_ease_of_use* 100):.1f}%')

Pick Up CES Score: 89.5%


Dengan Pick Up CES Score sebesar 89.5%, kemudahan penggunaan sistem tiket sudah cukup bagus. Ini menunjukkan bahwa sebagian besar pengguna merasa sistem ini cukup mudah digunakan, namun masih ada peluang untuk peningkatan agar mencapai yang lebih baik.










### NPS Score

To calculate the NPS score, first we must convert the **would_you_recommend** column into proper NPS Category based on the rating value:

* Promoter: Rating 9-10
* Passive: Rating 7-8
* Detractor: Rating < 7

In [ ]:
#Category NPS
nps_conditions = [
    responded_customer['likelihood_to_recommend'] >= 9,  # Promoter
    (responded_customer['likelihood_to_recommend'] >= 7) & (responded_customer['likelihood_to_recommend'] <= 8),  # Passive
    responded_customer['likelihood_to_recommend'] < 7  # Detractor
]

nps_values = ['Promoter', 'Passive', 'Detractor']

# Tambahkan kategori NPS ke dataframe
responded_customer['nps_category'] = np.select(nps_conditions, nps_values, default='Unknown')

# Hitung distribusi kategori NPS
responded_customer.value_counts('nps_category', normalize=True)


,proportion
nps_category,
Passive,0.48
Promoter,0.32
Detractor,0.20


Meskipun jumlah Promoter cukup baik, masih ada 20% Detractor, yang bisa berdampak pada reputasi layanan. Fokus perbaikan dapat diarahkan pada mengubah Passive menjadi Promoter dengan peningkatan fitur dan layanan pelanggan.


Calculate the NPS Score with the following formula

$$
NPS = \frac{Promoter - Detractor}{Total\ Survey\ Responded}
$$

In [ ]:
# NPS Score
nps_agg = responded_customer.value_counts('nps_category', normalize = True).reset_index()

nps_promoter = nps_agg[ nps_agg['nps_category'] == 'Promoter' ]['proportion'].item()
nps_detractor = nps_agg[ nps_agg['nps_category'] == 'Detractor' ]['proportion'].item()
nps_score = (nps_promoter - nps_detractor)

print(f'NPS Score: { (nps_score * 100):.1f}%')

NPS Score: 11.9%


Dengan NPS Score sebesar 11.9 %, tingkat rekomendasi pelanggan terhadap sistem tiket masuk dalam kategori "Average". Ini menunjukkan bahwa meskipun ada lebih banyak Promoter dibanding Detractor, tingkat loyalitas pelanggan masih belum terlalu kuat.

NPS score can be ranging from -100 (when all customers are detractor) to 100 (when all customers are promoter).

NPS Score can be classified into categories based on the following threshold:

- \>= 70: Excellent
- 50-69: Very Good
- 30-49: Good
- 0-29: Average
- \< 0: Poor

## Sentiment Analysis

Create a new dataframe with no blank overall_text.

In [ ]:
# Create new dataframe
df_clean = df.dropna(axis =0, subset = 'overall_text')[['id_survey', 'overall_text']].copy()

df_clean.head()

,id_survey,overall_text
1,T_00229,Customer tickets managements
2,T_04527,"After 6 months of using the Zoho desk, we shif..."
4,T_00644,Pros:Zendesk has always been one of the go-to ...
6,T_04682,It has been very useful so far to integrate mu...
8,T_01238,Pros:It's easy to use and very intuitive.We ha...


### Text Cleansing

In order to get more accurate sentiment, several text cleansing need to be done. However, in most of recent sentiment analysis models and algorithm, the only text cleansing needed are as follows:

* Clean double whitespace
* Clean URL/website
* Clean username (mostly in social media or digital text)

In [ ]:
import re

def cleansing_text(x):
  # clean double whitespace
  out_text = ' '.join(x.split())

  # clean url
  out_text = re.sub(r"http\S+|www\S+|https\S+", 'http', out_text)

  # clean username
  out_text = re.sub(r"@\S+", '@user', out_text)

  return(out_text)

cleansing_text("Great support team! The features are amazing. Check out @zoho for more info: https://zohodesk.com")

'Great support team! The features are amazing. Check out @user for more info: http'

In [ ]:
  `````````````````````````````A*(I# apply cleansing to review

df_clean['clean_text'] = df_clean['overall_text'].apply(cleansing_text)

df_clean.tail()

,id_survey,overall_text,clean_text
1457,T_01536,"IT was not prioritizing, tracking, resolving a...","IT was not prioritizing, tracking, resolving a..."
1458,T_00582,"Very useful, one-stop solution with related to...","Very useful, one-stop solution with related to..."
1459,T_01834,We have done quite well. We have already more ...,We have done quite well. We have already more ...
1460,T_03629,"Immensely positive, highly recommended custome...","Immensely positive, highly recommended custome..."
1461,T_01981,Pros:Easy to start with a basic free setup and...,Pros:Easy to start with a basic free setup and...


### Sentiment Analysis

Create a sentiment categories using algorithm of your own choice.

In [ ]:
%%capture
!pip install transformers

from transformers import pipeline

# Load sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Pastikan tidak ada NaN
df_clean['clean_text'] = df_clean['clean_text'].fillna("")

# Predict sentiment
transformer_results = sentiment_pipeline(df_clean['clean_text'].tolist())

# Simpan hasil prediksi dalam DataFrame
df_clean['sentiment_score'] =  transformer_results

# Ambil label sentimen dari hasil prediksi
df_clean['sentiment'] = df_clean['sentiment_score'].apply(lambda x: x['label'])

# Ubah label menjadi lebih mudah dibaca
sentiment_mapping = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

df_clean['sentiment'] = df_clean['sentiment'].map(sentiment_mapping)



In [ ]:

df_clean.head()

,id_survey,overall_text,clean_text,sentiment_score,sentiment
1,T_00229,Customer tickets managements,Customer tickets managements,"{'label': 'LABEL_1', 'score': 0.7328799366950989}",Neutral
2,T_04527,"After 6 months of using the Zoho desk, we shif...","After 6 months of using the Zoho desk, we shif...","{'label': 'LABEL_2', 'score': 0.9796792268753052}",Positive
4,T_00644,Pros:Zendesk has always been one of the go-to ...,Pros:Zendesk has always been one of the go-to ...,"{'label': 'LABEL_2', 'score': 0.7879295349121094}",Positive
6,T_04682,It has been very useful so far to integrate mu...,It has been very useful so far to integrate mu...,"{'label': 'LABEL_2', 'score': 0.9632710218429565}",Positive
8,T_01238,Pros:It's easy to use and very intuitive.We ha...,Pros:It's easy to use and very intuitive.We ha...,"{'label': 'LABEL_2', 'score': 0.8690706491470337}",Positive


Check the number of data by sentiment.

In [ ]:
# Number of Sentiment
df_clean.value_counts('sentiment',normalize = True)

,proportion
sentiment,
Positive,0.85
Neutral,0.13
Negative,0.03


Tingkat sentimen positif sangat dominan (85%), menandakan bahwa sistem tiket secara keseluruhan diterima dengan baik. Namun, untuk meningkatkan pengalaman pelanggan lebih lanjut, bisa dilakukan analisis mendalam pada ulasan negatif dan netral untuk mengidentifikasi area yang perlu perbaikan.

## Finalize Data for Reporting

Save the review data with NPS category and sentiment information to new csv for the dashboard.

In [ ]:
# Save Data
df_final = df.merge(df_clean[['id_survey', 'sentiment']],
                                  on = 'id_survey', how = 'left'
                                  )

df_final.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text,fill_survey,sentiment
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Responded,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements,Responded,Neutral
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif...",Responded,Positive
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Responded,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...,Responded,Positive


In [ ]:
file_output = data_path + 'ticket_system_final.csv'

df_final.to_csv(file_output, index = False)